[View in Colaboratory](https://colab.research.google.com/github/gauravbansal98/Save-and-Restore-a-model-in-tensorflow/blob/master/Save_and_restore_a_model.ipynb)

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("drive/Uploaded on github/mnist dataset using rnn", one_hot = True)

In [0]:
from tensorflow.python.ops import rnn, rnn_cell
tf.reset_default_graph()


In [0]:
rnn_size = 128
no_of_nodes_in_output_layer = 10
chunk_size = 56
no_of_chunk = 14
batch_size = 128
hm_epochs = 1

In [0]:
x = tf.placeholder(dtype = tf.float32, shape = [None, no_of_chunk, chunk_size], name = 'input_labels')
y = tf.placeholder(dtype = tf.float32, shape = [None, None], name = 'output_labels')

In [0]:
weights = { 'weights' : tf.Variable(tf.random_normal([rnn_size, no_of_nodes_in_output_layer])),
           'biases' : tf.Variable(tf.random_normal([1, no_of_nodes_in_output_layer])) }

In [0]:
def recurrent_neural_network(x):
  with tf.name_scope('Neural_Network'):
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, no_of_chunk, 0)
    cell = tf.nn.rnn_cell.BasicLSTMCell(rnn_size, name = 'LSTM_CELL')
    outputs, state = tf.nn.static_rnn(cell, x, dtype = tf.float32)

    output = tf.add(tf.matmul(outputs[-1], weights['weights']), weights['biases'])
    return output

In [0]:

prediction = recurrent_neural_network(x)

y_pred = tf.nn.softmax(prediction, name = "y_pred")

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y), name = "cost")

optimizer = tf.train.AdamOptimizer().minimize(cost)

saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  for i in range(hm_epochs):
    epoch_loss = 0
    for j in range(int(mnist.train.num_examples/batch_size)):
      epoch_x, epoch_y = mnist.train.next_batch(batch_size)
      epoch_x = epoch_x.reshape((batch_size,no_of_chunk,chunk_size))
      o, c = sess.run([optimizer, cost], feed_dict = {x : epoch_x, y : epoch_y})
      epoch_loss += c
    print('epoch' , i, 'completed out of ', hm_epochs, 'loss ', epoch_loss)

  saver.save(sess, './saved_model')
  
  correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

  
  accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

  print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape(-1,no_of_chunk,chunk_size), y:mnist.test.labels})) 

epoch 0 completed out of  1 loss  230.11198671162128
Accuracy: 0.9465


In [0]:
hm_epochs = 2

batch_size = 128

tf.reset_default_graph()

sess = tf.Session()

saver = tf.train.import_meta_graph('saved_model.meta')

print("meta graph loaded")

saver.restore(sess, tf.train.latest_checkpoint('./'))

graph = tf.get_default_graph()

x = graph.get_tensor_by_name("input_labels:0")

y = graph.get_tensor_by_name("output_labels:0")

last_layer_output = graph.get_tensor_by_name("y_pred:0")  

correct = tf.equal(tf.argmax(last_layer_output, 1), tf.argmax(y, 1))
  
accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

print('Accuracy:',sess.run(accuracy, feed_dict = {x:mnist.test.images.reshape(-1,no_of_chunk,chunk_size), y:mnist.test.labels})) 

sess.close()

meta graph loaded
INFO:tensorflow:Restoring parameters from ./saved_model
Accuracy: 0.9465
